In [1]:
#dependencies
from pathlib import Path
from matplotlib import pyplot as plt
from scipy import stats
import numpy as np
import pandas as pd

Main_csv_path = Path("Resources/18100205.csv")
Metadata_csv_path = Path("Resources/18100205_MetaData.csv")

Main_StatsCan_IndexDF = pd.read_csv(Main_csv_path, encoding="UTF-8")
Metadata_DF = pd.read_csv(Metadata_csv_path, encoding="UTF-8")

Main_StatsCan_IndexDF.tail()

,REF_DATE,GEO,DGUID,New housing price indexes,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
61075,2023-05,"Vancouver, British Columbia",2011S0503933,House only,"Index, 201612=100",347,units,0,v111955557,39.2,126.4,NaN,NaN,NaN,1
61076,2023-05,"Vancouver, British Columbia",2011S0503933,Land only,"Index, 201612=100",347,units,0,v111955558,39.3,121.0,E,NaN,NaN,1
61077,2023-05,"Victoria, British Columbia",2011S0503935,Total (house and land),"Index, 201612=100",347,units,0,v111955559,40.1,121.9,NaN,NaN,NaN,1
61078,2023-05,"Victoria, British Columbia",2011S0503935,House only,"Index, 201612=100",347,units,0,v111955560,40.2,127.9,NaN,NaN,NaN,1
61079,2023-05,"Victoria, British Columbia",2011S0503935,Land only,"Index, 201612=100",347,units,0,v111955561,40.3,110.6,E,NaN,NaN,1


In [2]:
Metadata_DF.head()

,Cube Title,Product Id,CANSIM Id,URL,Cube Notes,Archive Status,Frequency,Start Reference Period,End Reference Period,Total number of dimensions
"New housing price index, monthly",18100205,327-0056,https://www150.statcan.gc.ca/t1/tbl1/en/tv.act...,1;2;3;13;14,CURRENT - a cube available to the public and t...,Monthly,1981-01-01,2023-05-01,2.0,NaN
Dimension ID,Dimension name,Dimension Notes,Dimension Definitions,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Geography,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,New housing price indexes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Dimension ID,Member Name,Classification Code,Member ID,Parent Member ID,Terminated,Member Notes,Member Definitions,NaN,NaN,NaN


In [26]:
reduced_main_df = Main_StatsCan_IndexDF[["REF_DATE","GEO","New housing price indexes","UOM","VALUE","STATUS","DECIMALS"]]
reduced_main_df.head()

,REF_DATE,GEO,New housing price indexes,UOM,VALUE,STATUS,DECIMALS
0,1981-01,Canada,Total (house and land),"Index, 201612=100",38.2,NaN,1
1,1981-01,Canada,House only,"Index, 201612=100",36.1,NaN,1
2,1981-01,Canada,Land only,"Index, 201612=100",40.6,E,1
3,1981-01,Atlantic Region,Total (house and land),"Index, 201612=100",NaN,..,1
4,1981-01,Atlantic Region,House only,"Index, 201612=100",NaN,..,1


In [27]:
#import re
#print(re.split(r',',reduced_main_df["GEO"]))
Orig_Geo_series = pd.Series(reduced_main_df["GEO"])
#print(Orig_Geo_series)
Geo_Split = Orig_Geo_series.str.split(',')

Geo_City = Geo_Split.str.get(0) #access the first element for eventual "column1"

Geo_City

0                 Canada
1                 Canada
2                 Canada
3        Atlantic Region
4        Atlantic Region
              ...       
61075          Vancouver
61076          Vancouver
61077           Victoria
61078           Victoria
61079           Victoria
Name: GEO, Length: 61080, dtype: object

In [28]:
Geo_Prov = Geo_Split.str.get(1) #access the first element for eventual "column2"

Geo_Prov

0                      NaN
1                      NaN
2                      NaN
3                      NaN
4                      NaN
               ...        
61075     British Columbia
61076     British Columbia
61077     British Columbia
61078     British Columbia
61079     British Columbia
Name: GEO, Length: 61080, dtype: object

In [29]:
#Temp_DF = pd.concat(reduced_main_df,Geo_City )

Manual_Merge = pd.concat([Geo_City,Geo_Prov], axis=1) #axis=1 is using "column 1" to merge.

Manual_Merge

,GEO,GEO
0,Canada,NaN
1,Canada,NaN
2,Canada,NaN
3,Atlantic Region,NaN
4,Atlantic Region,NaN
...,...,...
61075,Vancouver,British Columbia
61076,Vancouver,British Columbia
61077,Victoria,British Columbia
61078,Victoria,British Columbia


In [ ]:
#next step1 is to merge the 2 VARs to the reduced_main_df ... and then remove the original "GEO" column.
#next step2 is to use pinned screen to "filter" our results to only what we want from the "STATUS" column.
#STATUS column filters are as follows.. REMOVE the "..", "E" and "x".... (keep blanks - "NaN" values) ... 
#NEW HOUSING PRICE INDEX .... column to be "Total (house and land)" ... SO REMOVE "House only" & "Land only"

#ALSO ... complete the same for "Geo_City" .. etc ... for "year+month" ... from the "REF_DATE" column ...
#GEO column ... we will "focus on keeping" ... which will be "Toronto, Vancouver and Montreal" ... 

#RENAME the columns to stuff that "makes common sense" ... for our presenting needs (which is dependent on the base prices).

In [ ]:
#SET UP the basepoints for the calculation with historical prices ...
#Create VARs to enable them to run against the cleaned data set ...

#Determine how to complete the analysis ... and fit into how they can be placed into slides.

#Confirm the types of "plots" ... we're confident in to speak to the story of the data analysis we're able to produce.
#Remember to comment that our $ costs are "approximations" ... due to needing the baseprice assumptions.

In [ ]:
#Slides Notes:
#Maria is testing
